In [2]:
import pandas as pd
import io
import msoffcrypto as ex
import numpy as np
import warnings
import re
global counta, countb, valsa, valsb, df, dfmerge

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 20)

HL7Field = ['MSH','PID','ORC','OBR','OBX','SPM','NTE','IN1','IN2','0009']
filedir = r"C:\Users\gmoye\FlatFileHL7\Extract_for_COVID_and_Influenza_20230503_1151.xlsx"
mapdir = r"C:\Users\gmoye\FlatFileHL7\FieldMap.xlsx"
hl7map = r"C:\Users\gmoye\FlatFileHL7\Maps.xlsx"
hometestmap = r"C:\Users\gmoye\FlatFileHL7\COVID-At-Home-Test-HL7-Guide-5-4-2022_0.xlsx"
RequiredFields = r"C:\Users\gmoye\FlatFileHL7\RequiredFields.xlsx"
Separators = r"C:\Users\gmoye\FlatFileHL7\Separators.xlsx"

pw = "test"

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    decrypted_workbook = io.BytesIO()
    with open(filedir, 'rb') as file:
        office_file = ex.OfficeFile(file)
        office_file.load_key(password=pw)
        office_file.decrypt(decrypted_workbook)
    
    all_sheets = pd.read_excel(decrypted_workbook, sheet_name=None)
    sheets = list(all_sheets.keys())

    df = pd.read_excel(decrypted_workbook, sheet_name=sheets[0], parse_dates=[['Order Date', 'Order Time'],['Result Date','Result Time'],['Collection Date','Collection Time']])
    
    count = 0
    for column, data in df.iteritems():
        count += 1
    
    df['record_no'] = np.arange(len(df))+1
    rcount = df.pop('record_no')
    df.insert(0, rcount.name, rcount)
    
    count = 0
    for col, vals in df[['Last Lab Results','Test Details']].items():
        count+=1
        count2 = 0
        for val in vals:
            count2+=1
            val = "'" + val + "'"
            val = "','".join(val.splitlines())
            val = str(val)
            df[f'{col}'].loc[df['record_no'] == count2] = val
    
    df2 = df.loc[((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    df = df.loc[~((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    
    df = df.drop(columns=['record_no'])
    df2 = df2.drop(columns=['record_no'])
    
    df2preserve = df2.copy(deep=True)
    
    df2 = df2[['Order ID', 'Last Lab Results','Test Details']]
    
    df2['record_no'] = np.arange(len(df2))+1
    rcount = df2.pop('record_no')
    df2.insert(0, rcount.name, rcount)
    
    for val in df2['record_no']:
        df2['record_no'].loc[df2['record_no'] == val] = str(val)+','+str(val)
    
    df3 = (df2.set_index(['Order ID'])).apply(lambda x: x.str.split(',').explode(['record_no','Last Lab Results','Test Details']))
    df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    
    dfsep = pd.merge(df2, df3, on = ['record_no'], how = 'inner')
    dfsep = dfsep.drop(columns=['Last Lab Results_x','Test Details_x'])
    df2preserve = df2preserve.drop(columns=['Last Lab Results','Test Details'])
    dfsep = dfsep.rename(columns={'Last Lab Results_y':'Last Lab Results','Test Details_y':'Test Details'})
    dfsep = pd.merge(dfsep, df2preserve, on = ['Order ID'], how = 'outer')


    df = pd.concat([df, dfsep]).reset_index(drop = True)

    df['Patient_ID_assigner'] = 'EPIC.ACC'
    df['Patient_ID_2_assigner'] = 'EPIC.MRN'
    df = df.rename(columns={'Order Date_Order Time':'Order_test_date','Result Date_Result Time':'Test_date','Collection Date_Collection Time':'Specimen_collection_date_time'})
    df = df.drop(columns=['record_no'])
    df.index.names = ['Field']

    column_names = df.columns.values.tolist()
    dfcolumns = pd.DataFrame(column_names)
    dfcolumns.rename(columns={0:'Field'}, inplace=True)

    dfmappings = pd.read_excel(mapdir)

    dfmappings = dfmappings.loc[dfmappings['Not Needed'] != 'X']

    dfdistinctmappings = dfmappings[['Field','NatELR Field ID', 'Segment', 'Not Needed','Option-Set needed']]

    dfmerge = pd.merge(dfcolumns, dfdistinctmappings, on = ['Field'], how = 'outer')
    dfmerge['NatELR Field ID'].loc[dfmerge['NatELR Field ID'].isna()] = dfmerge['Field']
    
    for col, val in dfmerge[['Field']].items():
        counta =-1
        for valsa in val:
            counta += 1
            valsa = str(valsa)
            for col, val in dfmerge[['NatELR Field ID']].items():
                countb = -1
                for valsb in val:
                    countb +=1
                    valsb = str(valsb)
                    if counta == countb:
                        df = df.rename(columns={f'{valsa}':f'{valsb}'}) 
        
    dfhl7 = pd.read_excel(hl7map)
    
    dfhl7home = pd.read_excel(hometestmap)
    
    dfhl7Separators = pd.read_excel(Separators)
    
    dflinesegments = pd.DataFrame(columns = ['Line Name','Line Code','Main Code','Sub Code','Segment', 'Field Name'])
    
    for vals in dfhl7Separators.values:
        linesubcode = None
        vals = vals[0]
        vals = vals.replace('-', '', 1).replace('.', '-', 1)
        Segment = vals.split(' ', 1)
        Segment1 = Segment[0]
        sep = '-'
        sep1 = '.'
        LineName = Segment1.split(sep, 1)[0]
        LineCode = Segment1.split(sep, 1)[1]
        CodeSplit = LineCode.split(sep1, 1)
        if len(CodeSplit) > 1:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = int(CodeSplit[1])
        else:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = '0'
            
        Segment2 = Segment[1].replace(' ', '', 1)
        count +=1
        dflinesegments = dflinesegments.append({'Line Name': LineName, 'Line Code': LineCode, 'Main Code': LineMainCode, 'Sub Code': LineSubCode, 'Segment': Segment1, 'Field Name': Segment2}, ignore_index=True)    
    
    dflinesegments = dflinesegments.sort_values(['Line Name', 'Main Code', 'Sub Code'], ascending=[1,1,1])
    
    pipe = '|'
    carrot = '^'
    dflinesegments['Separator'] = ''
    dflinesegments.loc[dflinesegments['Sub Code'] == '0', 'Separator'] = pipe
    dflinesegments.loc[dflinesegments['Sub Code'] != '0', 'Separator'] = carrot
    
    HL7Line = dfhl7home['Segment'].unique()
    
    dfhl7RequiredFields = pd.read_excel(RequiredFields)
    dfhl7Meaning = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna()]
    dfhl7Meaning = dfhl7Meaning.drop(dfhl7Meaning.columns[[1,2]], axis = 1).reset_index(drop = True)
    dfhl7RequiredFields = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna() == False]
    dfhl7RequiredFields = dfhl7RequiredFields.reset_index(drop = True)
    dfhl7RequiredFields['HL7 Element'] = dfhl7RequiredFields['HL7 Segment'].replace(regex=r"-\d{1,2}",value='').replace(regex=r"\.(.*)",value='')
    field = dfhl7RequiredFields.pop('HL7 Element')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
    dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
    dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
    dfhl7pivottest = dfhl7pivot.reset_index()
    dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()].dropna(axis=0, how='all').reset_index(drop=False)
    dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
    dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
    dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
    dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)
    
    
    
    lcol = list(dfhl7pivotreduction.columns)
    lcol.remove('Hardcode')
    
    vallist3 = []
    vallist4 = []
    vallist5 = []
    
    for val in dflinesegments['Line Name']:
        vallist3.append(val)
    for val in dflinesegments['Separator']:
        vallist4.append(val)
    for val in dflinesegments['Segment']:
        vallist5.append(val)
    
    listvals = dfmappings['NatELR Field ID'].values.tolist()
    listvals2 = dfmappings['Field Match - Short'].values.tolist()
    counta = -1
    for vals1 in listvals:
        counta += 1
        countb = -1
        for vals2 in listvals2:
            countb += 1
            if counta == countb:
                dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Element Name'] == vals2, 'HL7 Nat ELR Match'] = vals1
    
    field = dfhl7RequiredFields.pop('HL7 Nat ELR Match')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    loops = -1
    
    dfhl7RequiredFields = dfhl7RequiredFields.rename(columns={'HL7 Segment':'Segment'})
    #display(dfhl7RequiredFields)
    #display(dflinesegments)
    #display(dfhl7RequiredFields)
    #display(dflinesegments)
    
    dfforloopsfordays = pd.merge(dflinesegments, dfhl7RequiredFields, on = ['Segment'], how = 'left')
    #dfforloopsfordays = dfforloopsfordays.drop(columns=['HL7 Nat ELR Match','HL7 Element','Separator_y','HL7 Data Type','Data Type','Order'])
    dfforloopsfordays = dfforloopsfordays.rename(columns={'Separator_x':'Separator'})
    dfforloopsfordays['HL7 Element Name'] = dfforloopsfordays['HL7 Element Name'].fillna(dfforloopsfordays['Field Name'])
    dfforloopsfordays = dfforloopsfordays.drop(columns=['HL7 Element Name'])
    dfforloopsfordays = dfforloopsfordays.astype({'Main Code': int, 'Sub Code': int})
    dfforloopsfordays = dfforloopsfordays.sort_values(['Line Name','Main Code','Sub Code'], ascending=[1,1,1]).reset_index(drop = True)
    dfforloopsfordays = dfforloopsfordays.drop_duplicates(subset = 'Segment', keep = 'first')
    #display(dfhl7pivotreduction)
    pd.set_option('display.max_columns', None)
    pd.options.mode.chained_assignment = None
    pd.set_option('display.max_rows', None)
    #display(dfforloopsfordays)
    
    
    count = 0
    p1 = 'linesegment'
    varlist = []
    countalpha = 0
    for i in range(len(df)):
        countalpha += 1
        for val in HL7Field:
            linesegment = val
            segval = str(p1 + val)
            varlist.append(str(segval))
            line = f'{val}'
            count += 1
            count1 = 0
            for field, val1 in dfforloopsfordays[['HL7 Element','Segment','HL7 Nat ELR Match']].items():
                count1 += 1
                if count1 == 2:
                    count2 = -1
                    for val2 in val1:
                        count2 += 1
                        val2 = str(val2)
                        if val in val2:
                            try:
                                Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
                                 #'''MSH|^^^|^^0DSomeCLIA^^|^^^|^^^|^^||^^^|^^^^^|||||^^^^^^||^^^^|
                                 #   PID|^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^||^^^^^^^^^^^^^^|^^^^^^|^^^^^^^^^^^^^^||^^^^^^^^^^^^|^^^^^^^^^^^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^||^^^|^^^^^^^^^^|^^^^^^||||^^^^^^|^^^^^^|^^^^^^|^^||||current datetime^^^|^^^|^^^^^^|^^^^^^||^^^^^^|^^^^^^^^^|
                                 #   ORCRE||^^^^|^^^^|^^^^|||^^^^^^^^^^^^|^^|^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^|^^^^^^^^^^^^|^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^|^^^^^^^^^^|^401 North 9th Street^^^Bismarck^^ND^^58554^^^^^38015^^^^^^|^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|
                                 #   OBR|||||||||||||||||||||date of file||||P|||||||||||||||||||||||||||
                                 #   OBX|ST|||||year||||||P||||||||||||||||
                                 #   SPM|^^|^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^|||^^^^^^^^^|^^^^^^^^^|^^|^^|^^||^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^||^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|
                                 #   NTE||||
                                 #   IN1|^^^^^^|^^^^^^^^^^|^^^^^^^^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^^^^||^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^|||^^^||^^^^^^^^^^^^^^|^^^^^^|^^|^^^^^^^^^^^^^^||||||||||^^|^^^^^^^^^^^^^^^^^^^^^^^|||||||^^^^^^|^^^^^^||^^^^^^|^^^^^^|^^^^^^||^^^^^^^^^^^^^^|||||^^^^^^^^^^|||||
                                 #   IN2^^^^^^^^^^||^^^^^^^^^^^^^^^^^^^^^^^||||^^^^^^^^^^^^^^||^^^^^^^^^^^^^^||^^^^^^|||||||||||^^^^^^^^^^^^^^|||^^^^^^^^^^|^^^^^^^^^^||^^^^|^^^^|^^^|||^^^^^^|^^^^^^||^^^^^^|||^^^^^^|^^^^^^^^^^^^^^|^^^^^^|^^^^^^|^^^^^^||||^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^|||||^^^^^^^^^^^^|||^^^^^^^^^^|^^^^^^|^^^^^^^^^^^^|^^^^^^^^^^^^|^^^^^^||||^^^^^^^^^^|^^^^^^^^^^|^^^^^^|^^^^^^|
                                 #   0009'''
                                Hardcode = Hardcode.item()
                                if Hardcode != []:
                                    line += (Hardcode + vallist4[count2] + '')
                                else:
                                    continue
                            except:
                                pass
                            line += (vallist4[count2] + '')
                        segval = line
                        globals()[f'{p1}%s' % val] = segval
        
        dfpivredlist = dfhl7pivotreduction.columns.to_list()
    print(dfpivredlist, '\n')
    for val in dfpivredlist:
        if val != 'Hardcode':
            for vals in dfhl7pivotreduction[f'{val}']:
                vals = str(vals)
                if vals != 'nan':
                    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={f'{val}':f'{vals}'})
    
    dfpivredlist = dfhl7pivotreduction.columns.to_list()
    HC = pd.DataFrame()
    
    #rem = []
    #add = []
    #for val in enumerate(dfpivredlist):
    #    #print(val[1])
    #    val = str(val[1])
    #    if ' ' in val:
    #        rem.append(val)
    #        #dfpivredlist.remove(val)
    #        val = val.replace(' ', '')
    #        pattern = r'\[.*?\]'
    #        val = re.sub(pattern, '', val)
    #        #if val == 'PID-3.4.1':
    #            #val = 'PID-3.4'
    #        add.append(val)
            
    #for vals in rem:
    #    dfpivredlist.remove(vals)
       
    #for vals in add:
    #    dfpivredlist.append(vals)
    
    #add = [*set(add)]
    #print(add)
    
    #print(dfpivredlist, '\n')
    dfpivredlist = [*set(dfpivredlist)]
    #print(dfpivredlist, '\n')
    
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.4.1 [1]', 'PID-3.4').replace('PID-3.4.1 [2]', 'PID-3.4')
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.5 [1]', 'PID-3.5').replace('PID-3.5 [2]', 'PID-3.5')
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.4.1 [1]':'PID-3.4','PID-3.4.1 [2]':'PID-3.4'})
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.5 [1]':'PID-3.5','PID-3.5 [2]':'PID-3.5'})
    
    #display(dfhl7pivotreduction)
    
    hardlist = []
    counta = -1
    for vals in dfpivredlist:
        counta += 1
        if vals not in ['Hardcode','PID-3.4','PID-3.4.1','PID-3.5']:
            vals = str(vals)
            #print(vals)
            strcount = str(counta)
            hardval = str('HC'+strcount)
            hardlist.append(str(hardval))
            hardval = dfhl7pivotreduction.loc[dfhl7pivotreduction[f'{vals}'] == f'{vals}']
            #print(hardval)
            hardval = hardval.dropna(axis=1, how='all')
            hardval = hardval.reset_index(inplace = False, drop = True)
            globals()[str('HC'+strcount)] = hardval
    print('\n')
            #display(hardval)
            
        #df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    #print(hardlist)
    
    dfs['Hardcode'] = dfs['Hardcode'].astype(float)

    for val in hardlist:
        global values
        values = []
        dfs = eval(val)
        display(dfs)
        for (column, data) in dfs.iteritems():
            if column == 'Hardcode':
                HC = data.values[0]
        for (column, data) in dfs.iteritems():
            if column != 'Hardcode':
                value = data.values[0]
                values.append(value)
        for vals in values:
            print(HC, vals)
            dfforloopsfordays.loc[dfforloopsfordays['Segment'] == f'{vals}', 'Hardcode'] = HC
        #print(HC)
        #print(values)
    
    dfforloopsfordays.loc[dfforloopsfordays['Segment'] == 'PID-3.4.1', 'Hardcorde'] = 'EPIC.ACC or EPIC.MRN'
    dfforloopsfordays.loc[dfforloopsfordays['Segment'] == 'PID-3.5', 'Hardcorde'] = 'AN or MR'
    
    dfforloopsfordayshc = dfforloopsfordays.loc[dfforloopsfordays['Hardcode'].isna() == False]
    display(dfforloopsfordayshc)
        
        #if countalpha == 14:
        #    countbravo = 0
        #    for vals in varlist:
        #        countbravo += 1
        #        print(eval(vals))
        #        if countbravo == 10:
        #            countbravo = 0
        #            print('\n')
        #    print('\n')
            
            	

['Hardcode', 'MSH-10: Message Control ID', 'MSH-11: Processing ID', 'MSH-12: Version ID', 'MSH-15: Accept Acknowledgment Type', 'MSH-16: Application Acknowledgment Type', 'MSH-1: Field Separator', 'MSH-21: Message Profile Identifier', 'MSH-2: Encoding Characters', 'MSH-5: Receiving Application', 'MSH-6: Receiving Facility', 'MSH-9: Message Type', 'OBR-1: Set ID - OBR', 'ORC-1: Order Control', 'PID-1: Set ID - PID', 'PID-33: Last Update Date/Time', 'Date result released', 'Date this file was created', 'Format of the result', 'Name of the organization that created the patient ID', 'Name of the organization that created the patient ID #2', 'Order result status', 'Ordering Facility City', 'Ordering Facility State', 'Ordering facility county', 'Ordering facility street', 'Ordering facility zip code', 'Patient age units', 'Performing facility city', 'Performing facility county', 'Performing facility state', 'Performing facility street', 'Performing facility zip code', 'Report facil data sour

KeyError: 'PID-3.5 [1]'

In [16]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    dfpivredlist = dfhl7pivotreduction.columns.to_list()
    print(dfpivredlist, '\n')
    for val in dfpivredlist:
        if val != 'Hardcode':
            for vals in dfhl7pivotreduction[f'{val}']:
                vals = str(vals)
                if vals != 'nan':
                    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={f'{val}':f'{vals}'})
    
    dfpivredlist = dfhl7pivotreduction.columns.to_list()
    HC = pd.DataFrame()
    
    #rem = []
    #add = []
    #for val in enumerate(dfpivredlist):
    #    #print(val[1])
    #    val = str(val[1])
    #    if ' ' in val:
    #        rem.append(val)
    #        #dfpivredlist.remove(val)
    #        val = val.replace(' ', '')
    #        pattern = r'\[.*?\]'
    #        val = re.sub(pattern, '', val)
    #        #if val == 'PID-3.4.1':
    #            #val = 'PID-3.4'
    #        add.append(val)
            
    #for vals in rem:
    #    dfpivredlist.remove(vals)
       
    #for vals in add:
    #    dfpivredlist.append(vals)
    
    #add = [*set(add)]
    #print(add)
    
    #print(dfpivredlist, '\n')
    dfpivredlist = [*set(dfpivredlist)]
    #print(dfpivredlist, '\n')
    
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.4.1 [1]', 'PID-3.4').replace('PID-3.4.1 [2]', 'PID-3.4')
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.5 [1]', 'PID-3.5').replace('PID-3.5 [2]', 'PID-3.5')
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.4.1 [1]':'PID-3.4','PID-3.4.1 [2]':'PID-3.4'})
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.5 [1]':'PID-3.5','PID-3.5 [2]':'PID-3.5'})
    
    #display(dfhl7pivotreduction)
    
    hardlist = []
    counta = -1
    for vals in dfpivredlist:
        counta += 1
        if vals not in ['Hardcode','PID-3.4','PID-3.4.1','PID-3.5']:
            vals = str(vals)
            #print(vals)
            strcount = str(counta)
            hardval = str('HC'+strcount)
            hardlist.append(str(hardval))
            hardval = dfhl7pivotreduction.loc[dfhl7pivotreduction[f'{vals}'] == f'{vals}']
            #print(hardval)
            hardval = hardval.dropna(axis=1, how='all')
            hardval = hardval.reset_index(inplace = False, drop = True)
            globals()[str('HC'+strcount)] = hardval
    print('\n')
            #display(hardval)
            
        #df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    #print(hardlist)
    
    dfs['Hardcode'] = dfs['Hardcode'].astype(float)

    for val in hardlist:
        global values
        values = []
        dfs = eval(val)
        display(dfs)
        for (column, data) in dfs.iteritems():
            if column == 'Hardcode':
                HC = data.values[0]
        for (column, data) in dfs.iteritems():
            if column != 'Hardcode':
                value = data.values[0]
                values.append(value)
        for vals in values:
            print(HC, vals)
            dfforloopsfordays.loc[dfforloopsfordays['Segment'] == f'{vals}', 'Hardcode'] = HC
        #print(HC)
        #print(values)
    
    
    dfforloopsfordayshc = dfforloopsfordays.loc[dfforloopsfordays['Hardcode'].isna() == False]
    display(dfforloopsfordayshc)
        
        #countb = -1
        #for field, vals in dfs.items():
        #    countb += 1
        #    #print(len(dfs.columns))
        #    #field = str(field)
        #    print(field[0:])
        #    #if field == 'Hardcode':
        #    #    HC = str(vals[0])
        #    #if field != 'Hardcode':
        #    #    val = field
        #    #    print(HC, val)#, vals)
        ##dfforloopsfordays.loc[dfforloopsfordays['Segment'] == f'{val}', 'Hardcode'] = HC
        
        
        #countb = -1
        #for field, vals in dfs.items():
        #    countb += 1
        #    #print(vals[0], f': {countb}')
        #    if countb == 0:
        #        Hardcode = str(vals[0])
        #        print(Hardcode + ' ' + str(countb))
        #        #print(field)
        #    if countb > 0:
        #        field = str(field)
        #        print(field + ' ' + str(countb))
        #    dfforloopsfordays.loc[dfforloopsfordays['Segment'] == f'{field}', 'Hardcode'] = str(Hardcode)

['Hardcode', 'MSH-10', 'MSH-11', 'MSH-12', 'MSH-15', 'MSH-16', 'MSH-1', 'MSH-21', 'MSH-2', 'MSH-5', 'MSH-6', 'MSH-9', 'OBR-1', 'ORC-1', 'PID-1', 'PID-33', 'OBR-22', 'MSH-7', 'OBX-2', 'PID-3.4', 'PID-3.4', 'OBR-25', 'ORC-22.3', 'ORC-22.4', 'ORC-22.9', 'ORC-22.1', 'ORC-22.5', 'OBX-6', 'OBX-24.3', 'OBX-24.9', 'OBX-24.4', 'OBX-24.1', 'OBX-24.5', 'MSH-3', 'MSH-4.2', 'OBX-3.3', 'OBX-11', 'PID-3.5', 'PID-3.5'] 





ValueError: could not convert string to float: ''

In [53]:
dfforloopsfordayshc = dfforloopsfordays.loc[dfforloopsfordays['Hardcode'].isna() == False]
#display(dfhl7pivotreduction)
#print('\n')
display(dfforloopsfordayshc)
#print('\n')
#display(dfforloopsfordays)

,Line Name,Line Code,Main Code,Sub Code,Segment,Field Name,Separator,HL7 Nat ELR Match,HL7 Element,Separator_y,HL7 Data Type,Data Type,Required,Order,Hardcode
660,MSH,4.2,4,2,MSH-4.2,Universal Id,^,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0DSomeCLIA
705,OBR,1,1,0,OBR-1,Set ID - OBR,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
726,OBR,22,22,0,OBR-22,Results Rpt/Status Chng - Date/Time,|,NaN,OBR,|,NaN,NaN,NaN,26.0,date of file
729,OBR,25,25,0,OBR-25,Result Status,|,NaN,OBR,|,ID,Float,Required,27.0,P
756,OBX,2,2,0,OBX-2,Value Type,|,NaN,OBX,|,ID,Float,Required,29.0,ST
760,OBX,6,6,0,OBX-6,Units,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,year
765,OBX,11,11,0,OBX-11,Observation Result Status,|,NaN,OBX,|,ID,Float,Required,35.0,P
780,ORC,1,1,0,ORC-1,Order Control,|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RE
981,ORC,22.1,22,1,ORC-22.1,Street Address,^,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401 North 9th Street
983,ORC,22.3,22,3,ORC-22.3,City,^,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bismarck


In [ ]:
    display(dfhl7RequiredFields)
    display(dflinesegments)
    
    count = 0
    p1 = 'linesegment'
    varlist = []
    countalpha = 0
    
    for i in range(len(df)):
        countalpha += 1
        for val in HL7Field:
            linesegment = val
            segval = str(p1 + val)
            varlist.append(str(segval))
            line = f'{val}'
            count += 1
            count1 = 0
            for field, val1 in dfhl7RequiredFields[['HL7 Element','HL7 Segment','HL7 Nat ELR Match']].items():
                count1 += 1
                if count1 == 2:
                    count2 = -1
                    for val2 in val1:
                        count2 += 1
                        val2 = str(val2)
                        if val in val2:
                            try:
                                Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
                                Hardcode = Hardcode.item()
                                if Hardcode != []:
                                    line += (Hardcode + vallist4[count2] + '')
                                else:
                                    continue
                            except:
                                pass
                            line += (vallist4[count2] + '')
                        segval = line
                        globals()[f'{p1}%s' % val] = segval
                        
        if countalpha == 14:
            countbravo = 0
            for vals in varlist:
                countbravo += 1
                print(eval(vals))
                if countbravo == 10:
                    countbravo = 0
                    print('\n')
            print('\n')

In [10]:
print(dfhl7RequiredFields['HL7 Element Name'].values.tolist())

['Field Separator', 'Encoding Characters', 'Sending Application', 'Namespace ID', 'Universal ID', 'Universal ID Type', 'Sending Facility (i.e |lab name^CLIA code|CLIA)', 'Receiving Application', 'Receiving Facility', 'Date/Time of Message', 'Message Type', 'Message Control ID', 'Processing ID', 'Version ID', 'Set ID', 'Source of Comment', 'Comment', 'Comment Type', 'Filler Order Number', 'Universal Service Identifier', 'Observation Date Time', 'Specimen Source', 'Ordering Provider', 'ID Number', 'Family Name', 'Given Name', 'Results Rpt/Status Chng - Date/Time', 'Result Status', 'Set ID', 'Value Type', 'Observation Identifier', 'Observation Identifier - Identifier', 'Observation Identifier - Text', 'Observation Sub ID', 'Observation Value', 'Observation Result Status', 'Date Time Of The Observation', 'Producers Reference', 'Ordering Provider ID', 'Assigning Facility', 'Call Back Phone Number', 'Ordering Provider Telephone Number', 'Ordering Facility Name', 'Ordering Provider Facility O

In [350]:
import re
count = -1

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    dflinesegments = pd.DataFrame(columns = ['Line Name','Line Code','Main Code','Sub Code','Segment', 'Field Name'])
    #display(dflinesegments)
    for vals in dfhl7Separators.values:
        linesubcode = None
        vals = vals[0]
        vals = vals.replace('-', '', 1).replace('.', '-', 1)
        Segment = vals.split(' ', 1)
        Segment1 = Segment[0]
        sep = '-'
        sep1 = '.'
        LineName = Segment1.split(sep, 1)[0]
        LineCode = Segment1.split(sep, 1)[1]
        CodeSplit = LineCode.split(sep1, 1)
        if len(CodeSplit) > 1:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = int(CodeSplit[1])
        else:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = ''
            
        Segment2 = Segment[1].replace(' ', '', 1)
        count +=1
        dflinesegments = dflinesegments.append({'Line Name': LineName, 'Line Code': LineCode, 'Main Code': LineMainCode, 'Sub Code': LineSubCode, 'Segment': Segment1, 'Field Name': Segment2}, ignore_index=True)    
    
    dflinesegments = dflinesegments.sort_values(['Line Name', 'Main Code', 'Sub Code'], ascending=[1,1,1])
    
    pipe = '|'
    carrot = '^'
    dflinesegments['Separator'] = ''
    dflinesegments.loc[dflinesegments['Sub Code'] == '', 'Separator'] = pipe
    dflinesegments.loc[dflinesegments['Sub Code'] != '', 'Separator'] = carrot
    
    #dflinesegments.loc[dflinesegments['Sub Code'] is None, 'Separator'] = pipe
    #dflinesegments.loc[dflinesegments['Sub Code'] != None, 'Separator'] = carrot
    
    display(dflinesegments)
    

,Line Name,Line Code,Main Code,Sub Code,Segment,Field Name,Separator
0,IN1,1,1,,IN1-1,Set ID - IN1,|
71,IN1,2.1,2,1,IN1-2.1,Identifier,^
72,IN1,2.2,2,2,IN1-2.2,Text,^
73,IN1,2.3,2,3,IN1-2.3,Name Of Coding System,^
74,IN1,2.4,2,4,IN1-2.4,Alternate Identifier,^
75,IN1,2.5,2,5,IN1-2.5,Alternate Text,^
76,IN1,2.6,2,6,IN1-2.6,Name Of Alternate Coding System,^
70,IN1,2,2,,IN1-2,Insurance Plan ID,|
90,IN1,3.1,3,1,IN1-3.1,Id Number,^
92,IN1,3.2,3,2,IN1-3.2,Check Digit,^


In [264]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    count = 0
    p1 = 'linesegment'
    varlist = []
    countalpha = 0
    for i in range(len(df)):
        countalpha += 1
        for val in HL7Field:
            linesegment = val
            segval = str(p1 + val)
            varlist.append(str(segval))
            line = f'{val}'
            count += 1
            #print(val, '\n')
            count1 = 0
            for field, val1 in dfhl7RequiredFields[['HL7 Element','HL7 Segment','HL7 Nat ELR Match']].items():
                count1 += 1
                if count1 == 2:
                    count2 = -1
                    for val2 in val1:
                        count2 += 1
                        #print(val2 + ' ' + str(count2))
                        val2 = str(val2)
                        if val in val2:
                            try:
                                Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
                                Hardcode = Hardcode.item()
                                if Hardcode != []:
                                    line += (Hardcode + vallist4[count2] + '')
                                else:
                                    continue
                            except:
                                pass
                            line += (vallist4[count2] + '')
                        segval = line
                        globals()[f'{p1}%s' % val] = segval
                        
        if countalpha == len(df):
            countbravo = 0
            for vals in varlist:
                countbravo += 1
                print(eval(vals))
                if countbravo == 10:
                    countbravo = 0
                    print('\n')
            print('\n')
    
    count = 0
    for column, data in df.iteritems():
        count += 1
        print(count, column)
        #for val in data:
            #print(f'{column}:', val)
        

MSH|^~\&||CHI.EPIC||^^^|Maven^2.16.840.1.114222.4.3.4.34.1.1^ISO||NDHHS^2.16.840.1.113883.3.89.109.100.1.3^ISO||Timestamp of Message||ORU^R01^ORU_R01||generate from specID + a random ID ||P||2.5.1||
PID|^^^^|^^^^|||^|^^^^^^|^^
ORC^^|^|^||^^^^^
OBR|||||^^^date of file||P||^
OBX|ST|||^^||P||||
SPM|||^|||||
NTE||||
IN1^|
IN2|
0009|


MSH|^~\&||CHI.EPIC||^^^|Maven^2.16.840.1.114222.4.3.4.34.1.1^ISO||NDHHS^2.16.840.1.113883.3.89.109.100.1.3^ISO||Timestamp of Message||ORU^R01^ORU_R01||generate from specID + a random ID ||P||2.5.1||
PID|^^^^|^^^^|||^|^^^^^^|^^
ORC^^|^|^||^^^^^
OBR|||||^^^date of file||P||^
OBX|ST|||^^||P||||
SPM|||^|||||
NTE||||
IN1^|
IN2|
0009|


MSH|^~\&||CHI.EPIC||^^^|Maven^2.16.840.1.114222.4.3.4.34.1.1^ISO||NDHHS^2.16.840.1.113883.3.89.109.100.1.3^ISO||Timestamp of Message||ORU^R01^ORU_R01||generate from specID + a random ID ||P||2.5.1||
PID|^^^^|^^^^|||^|^^^^^^|^^
ORC^^|^|^||^^^^^
OBR|||||^^^date of file||P||^
OBX|ST|||^^||P||||
SPM|||^|||||
NTE||||
IN1^|
IN2|
0009|


M

In [232]:
#co = -1
#for field, val3 in dfhl7RequiredFields[['HL7 Nat ELR Match']].items():
#    #print(val3)
#    co += 1
#    #val3 = val3.loc[val3.isna()==False]
#    count = -1
#    if co == 0:
#        for val4 in val3:
#            #print(val4)
#            count += 1
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    count = -1
    const = 0
    for column, data in df.iteritems():
        for val in data:
            print(f'{column}:', val)

Order_test_date: 2022-11-17 15:36:21
Order_test_date: 2022-09-05 10:20:52
Order_test_date: 2022-06-21 08:16:18
Order_test_date: 2022-09-06 08:32:19
Order_test_date: 2022-06-03 10:17:47
Order_test_date: 2022-04-02 10:54:57
Order_test_date: 2022-08-09 12:26:52
Order_test_date: 2021-11-01 09:30:11
Order_test_date: 2021-11-19 09:41:00
Order_test_date: 2021-11-22 08:29:17
Order_test_date: 2019-09-16 12:32:24
Order_test_date: 2019-09-16 12:32:24
Order_test_date: 2022-04-02 10:54:57
Order_test_date: 2022-04-02 10:54:57
Test_date: 2022-11-17 15:59:12
Test_date: 2022-09-05 10:32:44
Test_date: 2022-06-21 08:50:56
Test_date: 2022-09-06 08:45:42
Test_date: 2022-06-03 10:26:37
Test_date: 2022-04-02 11:57:52
Test_date: 2022-08-09 12:58:50
Test_date: 2021-11-01 10:57:17
Test_date: 2021-11-19 11:12:05
Test_date: 2021-11-22 12:34:45
Test_date: 2019-09-16 14:52:00
Test_date: 2019-09-16 14:52:00
Test_date: 2022-04-02 11:38:06
Test_date: 2022-04-02 11:38:06
Specimen_collection_date_time: 2022-11-17 15:51:

In [272]:
print(len(df))

14


In [296]:
txt = "apple#banana#cherry#orange"
x = txt.split("#")
print(x)

['apple', 'banana', 'cherry', 'orange']


In [21]:
import pandas as pd
import io
import msoffcrypto as ex
import numpy as np
import warnings
import re
global counta, countb, valsa, valsb, df, dfmerge

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 20)

HL7Field = ['MSH','PID','ORC','OBR','OBX','SPM','NTE','IN1','IN2','0009']
filedir = r"C:\Users\gmoye\FlatFileHL7\Extract_for_COVID_and_Influenza_20230503_1151.xlsx"
mapdir = r"C:\Users\gmoye\FlatFileHL7\FieldMap.xlsx"
hl7map = r"C:\Users\gmoye\FlatFileHL7\Maps.xlsx"
hometestmap = r"C:\Users\gmoye\FlatFileHL7\COVID-At-Home-Test-HL7-Guide-5-4-2022_0.xlsx"
RequiredFields = r"C:\Users\gmoye\FlatFileHL7\RequiredFields.xlsx"
Separators = r"C:\Users\gmoye\FlatFileHL7\Separators.xlsx"

pw = "test"

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', pd.errors.PerformanceWarning)
    warnings.simplefilter('ignore', FutureWarning)

    decrypted_workbook = io.BytesIO()
    with open(filedir, 'rb') as file:
        office_file = ex.OfficeFile(file)
        office_file.load_key(password=pw)
        office_file.decrypt(decrypted_workbook)
    
    all_sheets = pd.read_excel(decrypted_workbook, sheet_name=None)
    sheets = list(all_sheets.keys())

    df = pd.read_excel(decrypted_workbook, sheet_name=sheets[0], parse_dates=[['Order Date', 'Order Time'],['Result Date','Result Time'],['Collection Date','Collection Time']])
    
    count = 0
    for column, data in df.iteritems():
        count += 1
    
    df['record_no'] = np.arange(len(df))+1
    rcount = df.pop('record_no')
    df.insert(0, rcount.name, rcount)
    
    count = 0
    for col, vals in df[['Last Lab Results','Test Details']].items():
        count+=1
        count2 = 0
        for val in vals:
            count2+=1
            val = "'" + val + "'"
            val = "','".join(val.splitlines())
            val = str(val)
            df[f'{col}'].loc[df['record_no'] == count2] = val
    
    df2 = df.loc[((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    df = df.loc[~((df['Last Lab Results'].str.contains(',')) | (df['Test Details'].str.contains(',')))]
    
    df = df.drop(columns=['record_no'])
    df2 = df2.drop(columns=['record_no'])
    
    df2preserve = df2.copy(deep=True)
    
    df2 = df2[['Order ID', 'Last Lab Results','Test Details']]
    
    df2['record_no'] = np.arange(len(df2))+1
    rcount = df2.pop('record_no')
    df2.insert(0, rcount.name, rcount)
    
    for val in df2['record_no']:
        df2['record_no'].loc[df2['record_no'] == val] = str(val)+','+str(val)
    
    df3 = (df2.set_index(['Order ID'])).apply(lambda x: x.str.split(',').explode(['record_no','Last Lab Results','Test Details']))
    df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    
    dfsep = pd.merge(df2, df3, on = ['record_no'], how = 'inner')
    dfsep = dfsep.drop(columns=['Last Lab Results_x','Test Details_x'])
    df2preserve = df2preserve.drop(columns=['Last Lab Results','Test Details'])
    dfsep = dfsep.rename(columns={'Last Lab Results_y':'Last Lab Results','Test Details_y':'Test Details'})
    dfsep = pd.merge(dfsep, df2preserve, on = ['Order ID'], how = 'outer')


    df = pd.concat([df, dfsep]).reset_index(drop = True)

    df['Patient_ID_assigner'] = 'EPIC.ACC'
    df['Patient_ID_2_assigner'] = 'EPIC.MRN'
    df = df.rename(columns={'Order Date_Order Time':'Order_test_date','Result Date_Result Time':'Test_date','Collection Date_Collection Time':'Specimen_collection_date_time'})
    df = df.drop(columns=['record_no'])
    df.index.names = ['Field']

    column_names = df.columns.values.tolist()
    dfcolumns = pd.DataFrame(column_names)
    dfcolumns.rename(columns={0:'Field'}, inplace=True)

    dfmappings = pd.read_excel(mapdir)

    dfmappings = dfmappings.loc[dfmappings['Not Needed'] != 'X']

    dfdistinctmappings = dfmappings[['Field','NatELR Field ID', 'Segment', 'Not Needed','Option-Set needed']]

    dfmerge = pd.merge(dfcolumns, dfdistinctmappings, on = ['Field'], how = 'outer')
    dfmerge['NatELR Field ID'].loc[dfmerge['NatELR Field ID'].isna()] = dfmerge['Field']
    
    for col, val in dfmerge[['Field']].items():
        counta =-1
        for valsa in val:
            counta += 1
            valsa = str(valsa)
            for col, val in dfmerge[['NatELR Field ID']].items():
                countb = -1
                for valsb in val:
                    countb +=1
                    valsb = str(valsb)
                    if counta == countb:
                        df = df.rename(columns={f'{valsa}':f'{valsb}'}) 
        
    dfhl7 = pd.read_excel(hl7map)
    
    dfhl7home = pd.read_excel(hometestmap)
    
    dfhl7Separators = pd.read_excel(Separators)
    
    dflinesegments = pd.DataFrame(columns = ['Line Name','Line Code','Main Code','Sub Code','Segment', 'Field Name'])
    
    for vals in dfhl7Separators.values:
        linesubcode = None
        vals = vals[0]
        vals = vals.replace('-', '', 1).replace('.', '-', 1)
        Segment = vals.split(' ', 1)
        Segment1 = Segment[0]
        sep = '-'
        sep1 = '.'
        LineName = Segment1.split(sep, 1)[0]
        LineCode = Segment1.split(sep, 1)[1]
        CodeSplit = LineCode.split(sep1, 1)
        if len(CodeSplit) > 1:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = int(CodeSplit[1])
        else:
            LineMainCode = int(CodeSplit[0])
            LineSubCode = '0'
            
        Segment2 = Segment[1].replace(' ', '', 1)
        count +=1
        dflinesegments = dflinesegments.append({'Line Name': LineName, 'Line Code': LineCode, 'Main Code': LineMainCode, 'Sub Code': LineSubCode, 'Segment': Segment1, 'Field Name': Segment2}, ignore_index=True)    
    
    dflinesegments = dflinesegments.sort_values(['Line Name', 'Main Code', 'Sub Code'], ascending=[1,1,1])
    
    pipe = '|'
    carrot = '^'
    dflinesegments['Separator'] = ''
    dflinesegments.loc[dflinesegments['Sub Code'] == '0', 'Separator'] = pipe
    dflinesegments.loc[dflinesegments['Sub Code'] != '0', 'Separator'] = carrot
    
    HL7Line = dfhl7home['Segment'].unique()
    
    dfhl7RequiredFields = pd.read_excel(RequiredFields)
    dfhl7Meaning = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna()]
    dfhl7Meaning = dfhl7Meaning.drop(dfhl7Meaning.columns[[1,2]], axis = 1).reset_index(drop = True)
    dfhl7RequiredFields = dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Segment'].isna() == False]
    dfhl7RequiredFields = dfhl7RequiredFields.reset_index(drop = True)
    dfhl7RequiredFields['HL7 Element'] = dfhl7RequiredFields['HL7 Segment'].replace(regex=r"-\d{1,2}",value='').replace(regex=r"\.(.*)",value='')
    field = dfhl7RequiredFields.pop('HL7 Element')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    dfhl7pivot = dfhl7.pivot(index=['Variable Name','Instructions'], columns=['Hardcode'], values=['HL7 Field'])#.reset_index(drop=False)
    dfhl7pivot.columns = dfhl7pivot.columns.droplevel(0)
    dfhl7pivot = dfhl7pivot.dropna(axis=1, how='all')
    dfhl7pivottest = dfhl7pivot.reset_index()
    dfhl7pivotreduction = dfhl7pivot.loc[:, dfhl7pivot.columns.notna()].dropna(axis=0, how='all').reset_index(drop=False)
    dfhl7pivotreduction["Variable Name"] = dfhl7pivotreduction["Variable Name"].fillna(dfhl7pivotreduction["Instructions"])
    dfhl7pivotreduction = dfhl7pivotreduction.set_index(["Instructions", "Variable Name"]).T#.reset_index(drop=False)
    dfhl7pivotreduction.columns = dfhl7pivotreduction.columns.droplevel(0)
    dfhl7pivotreduction = dfhl7pivotreduction.reset_index(drop=False)
    
    
    
    lcol = list(dfhl7pivotreduction.columns)
    lcol.remove('Hardcode')
    
    vallist3 = []
    vallist4 = []
    vallist5 = []
    
    for val in dflinesegments['Line Name']:
        vallist3.append(val)
    for val in dflinesegments['Separator']:
        vallist4.append(val)
    for val in dflinesegments['Segment']:
        vallist5.append(val)
    
    listvals = dfmappings['NatELR Field ID'].values.tolist()
    listvals2 = dfmappings['Field Match - Short'].values.tolist()
    counta = -1
    for vals1 in listvals:
        counta += 1
        countb = -1
        for vals2 in listvals2:
            countb += 1
            if counta == countb:
                dfhl7RequiredFields.loc[dfhl7RequiredFields['HL7 Element Name'] == vals2, 'HL7 Nat ELR Match'] = vals1
    
    field = dfhl7RequiredFields.pop('HL7 Nat ELR Match')
    dfhl7RequiredFields.insert(1, field.name, field)
    
    loops = -1
    
    dfhl7RequiredFields = dfhl7RequiredFields.rename(columns={'HL7 Segment':'Segment'})
    #display(dfhl7RequiredFields)
    #display(dflinesegments)
    #display(dfhl7RequiredFields)
    #display(dflinesegments)
    
    dfforloopsfordays = pd.merge(dflinesegments, dfhl7RequiredFields, on = ['Segment'], how = 'left')
    #dfforloopsfordays = dfforloopsfordays.drop(columns=['HL7 Nat ELR Match','HL7 Element','Separator_y','HL7 Data Type','Data Type','Order'])
    dfforloopsfordays = dfforloopsfordays.rename(columns={'Separator_x':'Separator'})
    dfforloopsfordays['HL7 Element Name'] = dfforloopsfordays['HL7 Element Name'].fillna(dfforloopsfordays['Field Name'])
    dfforloopsfordays = dfforloopsfordays.drop(columns=['HL7 Element Name'])
    dfforloopsfordays = dfforloopsfordays.astype({'Main Code': int, 'Sub Code': int})
    dfforloopsfordays = dfforloopsfordays.sort_values(['Line Name','Main Code','Sub Code'], ascending=[1,1,1]).reset_index(drop = True)
    dfforloopsfordays = dfforloopsfordays.drop_duplicates(subset = 'Segment', keep = 'first')
    #display(dfhl7pivotreduction)
    pd.set_option('display.max_columns', None)
    pd.options.mode.chained_assignment = None
    pd.set_option('display.max_rows', None)
    #display(dfforloopsfordays)
    
    
    count = 0
    p1 = 'linesegment'
    varlist = []
    countalpha = 0
    for i in range(len(df)):
        countalpha += 1
        for val in HL7Field:
            linesegment = val
            segval = str(p1 + val)
            varlist.append(str(segval))
            line = f'{val}'
            count += 1
            count1 = 0
            for field, val1 in dfforloopsfordays[['HL7 Element','Segment','HL7 Nat ELR Match']].items():
                count1 += 1
                if count1 == 2:
                    count2 = -1
                    for val2 in val1:
                        count2 += 1
                        val2 = str(val2)
                        if val in val2:
                            try:
                                Hardcode = dfhl7pivotreduction['Hardcode'].loc[(dfhl7pivotreduction.eq(f'{val2}').any(1)) & (dfhl7pivotreduction['Hardcode'] != '|')]
                                 #'''MSH|^^^|^^0DSomeCLIA^^|^^^|^^^|^^||^^^|^^^^^|||||^^^^^^||^^^^|
                                 #   PID|^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^||^^^^^^^^^^^^^^|^^^^^^|^^^^^^^^^^^^^^||^^^^^^^^^^^^|^^^^^^^^^^^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^||^^^|^^^^^^^^^^|^^^^^^||||^^^^^^|^^^^^^|^^^^^^|^^||||current datetime^^^|^^^|^^^^^^|^^^^^^||^^^^^^|^^^^^^^^^|
                                 #   ORCRE||^^^^|^^^^|^^^^|||^^^^^^^^^^^^|^^|^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^^^^^^|^^^^^^^^^^^^|^^|^^^^^^|^^^^^^|^^^^^^|^^^^^^^^^^^^^^^^^^^^^^^|^^^^^^|^^^^^^^^^^|^401 North 9th Street^^^Bismarck^^ND^^58554^^^^^38015^^^^^^|^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|
                                 #   OBR|||||||||||||||||||||date of file||||P|||||||||||||||||||||||||||
                                 #   OBX|ST|||||year||||||P||||||||||||||||
                                 #   SPM|^^|^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^|||^^^^^^^^^|^^^^^^^^^|^^|^^|^^||^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|^^||^^^^^^^^^|^^^^^^^^^|^^^^^^^^^|
                                 #   NTE||||
                                 #   IN1|^^^^^^|^^^^^^^^^^|^^^^^^^^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^^^|^^^^^^^^^^^^||^^^^^^^^^^|^^^^^^^^^^|^^^^^^^^^^|||^^^||^^^^^^^^^^^^^^|^^^^^^|^^|^^^^^^^^^^^^^^||||||||||^^|^^^^^^^^^^^^^^^^^^^^^^^|||||||^^^^^^|^^^^^^||^^^^^^|^^^^^^|^^^^^^||^^^^^^^^^^^^^^|||||^^^^^^^^^^|||||
                                 #   IN2^^^^^^^^^^||^^^^^^^^^^^^^^^^^^^^^^^||||^^^^^^^^^^^^^^||^^^^^^^^^^^^^^||^^^^^^|||||||||||^^^^^^^^^^^^^^|||^^^^^^^^^^|^^^^^^^^^^||^^^^|^^^^|^^^|||^^^^^^|^^^^^^||^^^^^^|||^^^^^^|^^^^^^^^^^^^^^|^^^^^^|^^^^^^|^^^^^^||||^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^||^^^^^^^^^^^^^^|^^^^^^^^^^^^|||||^^^^^^^^^^^^|||^^^^^^^^^^|^^^^^^|^^^^^^^^^^^^|^^^^^^^^^^^^|^^^^^^||||^^^^^^^^^^|^^^^^^^^^^|^^^^^^|^^^^^^|
                                 #   0009'''
                                Hardcode = Hardcode.item()
                                if Hardcode != []:
                                    line += (Hardcode + vallist4[count2] + '')
                                else:
                                    continue
                            except:
                                pass
                            line += (vallist4[count2] + '')
                        segval = line
                        globals()[f'{p1}%s' % val] = segval
        
        dfpivredlist = dfhl7pivotreduction.columns.to_list()
        
    print(dfpivredlist, '\n')
    for val in dfpivredlist:
        if val != 'Hardcode':
            for vals in dfhl7pivotreduction[f'{val}']:
                vals = str(vals)
                if vals != 'nan':
                    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={f'{val}':f'{vals}'})
    
    dfpivredlist = dfhl7pivotreduction.columns.to_list()
    HC = pd.DataFrame()
    
    rem = []
    add = []
    for val in enumerate(dfpivredlist):
        #print(val[1])
        val = str(val[1])
        if ' ' in val:
            rem.append(val)
            #dfpivredlist.remove(val)
            val = val.replace(' ', '')
            pattern = r'\[.*?\]'
            val = re.sub(pattern, '', val)
            #if val == 'PID-3.4.1':
                #val = 'PID-3.4'
            add.append(val)
           
    for vals in rem:
        dfpivredlist.remove(vals)
      
    for vals in add:
        dfpivredlist.append(vals)
    
    #print(dfpivredlist, '\n')
    dfpivredlist = [*set(dfpivredlist)]
    #print(dfpivredlist, '\n')
    
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.4.1 [1]', 'PID-3.4').replace('PID-3.4.1 [2]', 'PID-3.4')
    dfhl7pivotreduction = dfhl7pivotreduction.replace('PID-3.5 [1]', 'PID-3.5').replace('PID-3.5 [2]', 'PID-3.5')
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.4.1 [1]':'PID-3.4','PID-3.4.1 [2]':'PID-3.4'})
    dfhl7pivotreduction = dfhl7pivotreduction.rename(columns={'PID-3.5 [1]':'PID-3.5','PID-3.5 [2]':'PID-3.5'})
    
    #display(dfhl7pivotreduction)
    
    hardlist = []
    counta = -1
    for vals in dfpivredlist:
        counta += 1
        if vals not in ['Hardcode','PID-3.4','PID-3.4.1','PID-3.5']:
            vals = str(vals)
            #print(vals)
            strcount = str(counta)
            hardval = str('HC'+strcount)
            hardlist.append(str(hardval))
            hardval = dfhl7pivotreduction.loc[dfhl7pivotreduction[f'{vals}'] == f'{vals}']
            #print(hardval)
            hardval = hardval.dropna(axis=1, how='all')
            hardval = hardval.reset_index(inplace = False, drop = True)
            globals()[str('HC'+strcount)] = hardval
    print('\n')
            #display(hardval)
            
        #df2['record_no'] = df2['record_no'].str.replace(r'\,\d','').astype(object)
    #print(hardlist)
    
    dfs['Hardcode'] = dfs['Hardcode'].astype(float)

    for val in hardlist:
        global values
        values = []
        dfs = eval(val)
        display(dfs)
        for (column, data) in dfs.iteritems():
            if column == 'Hardcode':
                HC = data.values[0]
        for (column, data) in dfs.iteritems():
            if column != 'Hardcode':
                value = data.values[0]
                values.append(value)
        for vals in values:
            print(HC, vals)
            dfforloopsfordays.loc[dfforloopsfordays['Segment'] == f'{vals}', 'Hardcode'] = HC
        #print(HC)
        #print(values)
    
    
    dfforloopsfordayshc = dfforloopsfordays.loc[dfforloopsfordays['Hardcode'].isna() == False]
    display(dfforloopsfordayshc)
        
        #if countalpha == 14:
        #    countbravo = 0
        #    for vals in varlist:
        #        countbravo += 1
        #        print(eval(vals))
        #        if countbravo == 10:
        #            countbravo = 0
        #            print('\n')
        #    print('\n')

['Hardcode', 'MSH-10: Message Control ID', 'MSH-11: Processing ID', 'MSH-12: Version ID', 'MSH-15: Accept Acknowledgment Type', 'MSH-16: Application Acknowledgment Type', 'MSH-1: Field Separator', 'MSH-21: Message Profile Identifier', 'MSH-2: Encoding Characters', 'MSH-5: Receiving Application', 'MSH-6: Receiving Facility', 'MSH-9: Message Type', 'OBR-1: Set ID - OBR', 'ORC-1: Order Control', 'PID-1: Set ID - PID', 'PID-33: Last Update Date/Time', 'Date result released', 'Date this file was created', 'Format of the result', 'Name of the organization that created the patient ID', 'Name of the organization that created the patient ID #2', 'Order result status', 'Ordering Facility City', 'Ordering Facility State', 'Ordering facility county', 'Ordering facility street', 'Ordering facility zip code', 'Patient age units', 'Performing facility city', 'Performing facility county', 'Performing facility state', 'Performing facility street', 'Performing facility zip code', 'Report facil data sour

ValueError: could not convert string to float: ''